# MSc Thesis - Hugo Stam

### Example 03 - Offshore Port System - Bridge

<img src="Figures/system_layout_alternative_3.2.png" style="width:1000px">

##### Investment Strategy Simulation - Adaptive terminal planning in the light of an uncertain future

* [**0. Prepare inputs:**](#0.-Prepare-inputs)<br>
   * [**0.1 Generate demand forecast scenario:**](#0.1-Generate-demand-forecast-scenario)<br>
   * [**0.2 Generate vessels:**](#0.2-Generate-vessels)<br>
* [**1. Instatiate system components:**](#1.-Instatiate-system-components)<br>
* [**2. Start simulation:**](#2.-Strart-simulation)<br>
* [**3. Report all elements:**](#3.-Report-all-elements)<br>

#### notes:


In [1]:
# packages for data handling
import numpy as np
import pandas as pd
import statistics as st

# packages related to time, space and id
import datetime, time
import platform
import random
from datetime import timedelta

# you need these dependencies packages related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# packages for figures
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
# sns.set(style="ticks")
# sns.set(style="darkgrid", palette = "pastel")
# sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.5})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# OpenTISim package
from opentisim import container_objects
from opentisim import container_defaults
from opentisim import container_system_offshore_bridge

# OpenCLSim package
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# Additional import to save the initialisation of the simulation
import openclsim.savesim as savesim

# Pretty-print a Python object to a stream
from pprint import pprint

# Jupyter Widgets
import ipywidgets as widgets

# 
from IPython.display import HTML

# Scroll to specific cell
from jupyter_helpers.utilities import scroll_to_current_cell

# 0. Prepare inputs

In [2]:
# iPython Widgests
annual_demand = widgets.IntSlider(
    value=1000000, min=500000, max=1500000, step=50000,
    description='Demand:', orientation='horizontal',
    readout=True, readout_format='d')

offshore_onshore_distance = widgets.IntSlider(
    value=40, min=20, max=60, step=20,
    description='Distance:', orientation='horizontal',
    readout=True, readout_format='d')

container_ship = widgets.Dropdown(
    options=[('Post Panamax I (6,000 TEU)', 6000), 
             ('New-Panamax (12,500 TEU)', 12500), 
             ('ULCS (21,000 TEU)', 21000)],
    value=12500, 
    description='OGV:')

truck = widgets.Dropdown(
    options=[('Small (2 TEU)', 2), 
             ('Medium (2.5 TEU)', 2.5), 
             ('Large (3 TEU)', 3)],
    value=2, 
    description='Truck:')

life_cycle = widgets.IntSlider(
    value=10, min=5, max=30, step=5,
    description='Life cycle:', orientation='horizontal',
    readout=True, readout_format='d')

print('Input Values')
display(annual_demand, offshore_onshore_distance, container_ship, truck, life_cycle)

Input Values


IntSlider(value=1000000, description='Demand:', max=1500000, min=500000, step=50000)

IntSlider(value=40, description='Distance:', max=60, min=20, step=20)

Dropdown(description='OGV:', index=1, options=(('Post Panamax I (6,000 TEU)', 6000), ('New-Panamax (12,500 TEU…

Dropdown(description='Truck:', options=(('Small (2 TEU)', 2), ('Medium (2.5 TEU)', 2.5), ('Large (3 TEU)', 3))…

IntSlider(value=10, description='Life cycle:', max=30, min=5, step=5)

In [3]:
offshore_distance = offshore_onshore_distance.value
demand = annual_demand.value
design_container_ship = container_ship.value
design_truck = truck.value
lifecycle = life_cycle.value

print('The offshore distance:', offshore_distance, 'km')
print('The annual demand:', demand, 'TEU')
print('The design container ship capacity:', design_container_ship, 'TEU')
print('The design barge capacity:', design_truck, 'TEU')
print('The design life cycle:', lifecycle, 'years')

avg_parcel = design_container_ship / 8

startyear = 2020

The offshore distance: 40 km
The annual demand: 1000000 TEU
The design container ship capacity: 12500 TEU
The design barge capacity: 2 TEU
The design life cycle: 10 years


## 0.1 Generate demand forecast scenario

In [4]:
# total_ocean_transport = 1000000  # TEU
# years = list(range(startyear, startyear+lifecycle))
# demand_list = []

# for year in years:
#     if year < 2025:
#         demand.append(total_ocean_transport)
#     elif year < 2030:
#         demand.append(total_ocean_transport * 1.5)
#     else:
#         demand.append(total_ocean_transport * 2.0)

# plt.plot(years, demand, 'o')
# plt.ylabel('Demand [TEU]')

In [5]:
years = list(range(startyear, startyear + lifecycle))
        
scenario_data={'year': years, 'volume': demand}

# instantiate Commodity objects, the inputs for the Commodity class
if design_container_ship == 6000:
    container_defaults.container_data['post_panamax_I_perc'] = 100
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 0

if design_container_ship == 12500:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 100
    container_defaults.container_data['ULCS_perc'] = 0
    
if design_container_ship == 21000:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 100
    
print(container_defaults.container_data)

# ladens
container = container_objects.Commodity(**container_defaults.container_data)
container.scenario_data = pd.DataFrame(data=scenario_data)
# print(container.scenario_data)

# combine
demand = [container]

{'name': 'Laden', 'handling_fee': 150, 'fully_cellular_perc': 0, 'panamax_perc': 0, 'panamax_max_perc': 0, 'post_panamax_I_perc': 0, 'post_panamax_II_perc': 0, 'new_panamax_perc': 100, 'VLCS_perc': 0, 'ULCS_perc': 0}


## 0.2 Generate vessels

In [6]:
# instantiate vessels
fully_cellular = container_objects.Vessel(**container_defaults.fully_cellular_data)
panamax = container_objects.Vessel(**container_defaults.panamax_data)
panamax_max = container_objects.Vessel(**container_defaults.panamax_max_data)
post_panamax_I = container_objects.Vessel(**container_defaults.post_panamax_I_data)
post_panamax_II = container_objects.Vessel(**container_defaults.post_panamax_II_data)
new_panamax = container_objects.Vessel(**container_defaults.new_panamax_data)
VLCS = container_objects.Vessel(**container_defaults.VLCS_data)
ULCS = container_objects.Vessel(**container_defaults.ULCS_data)

vessels = [fully_cellular, panamax, panamax_max, post_panamax_I, post_panamax_II, new_panamax, VLCS, ULCS] 

# 1. Instatiate system components

## 1.1 Terminal system

Specify the variables

In [7]:
OffshoreBridge = container_system_offshore_bridge.System(
                 startyear = startyear,
                 lifecycle = lifecycle,
                 stack_equipment = 'rs',
                 laden_stack = 'rs',
                 elements = demand + vessels,
                 crane_type_defaults = container_defaults.sts_crane_data,
                 offshore_distance = offshore_distance,
                 bathymetry_factor = 0.5,
                 allowable_berth_occupancy = 0.50,
                 transhipment_ratio = 0.0,
                 debug = True)

# 2. Start simulation

This method automatically generates investment decisions, parametrically derived from demand trends and a number of investment triggers.

Apply frame of reference style decisions while stepping through each year of the terminal lifecycle and check if investment is needed (in light of strategic objective, operational objective, QSC, decision recipe, intervention method):
    1. for each year evaluate the demand of each commodity (see 0.1 Demand forecast scenario)
    2. for each year evaluate the various investment decisions
    3. for each year calculate the energy costs (requires insight in realized demands)
    4. for each year calculate the demurrage costs (requires insight in realized demands)
    5. for each year calculate terminal revenues
    6. collect all cash flows (capex, opex, revenues)
    7. calculate PV's and aggregate to NPV

In [8]:
OffshoreBridge.simulate()


Offshore Port System - Bridge Connection

Below, the various investment decisions are evaluated for the year 2020.

Simulate year: 2020
  Total vessel calls: 640
  Fully cellular calls: 0
  Panamax calls: 0
  Panamax max calls: 0
  Post Panamax I calls: 0
  Post Panamax II calls: 0
  New Panamax calls: 640
  VLCS calls: 0
  ULCS calls: 0
  Total cargo volume: 1000000
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned

  Start analysis:
     Berth occupancy planned (@ start of year): inf
     Berth occupancy online  (@ start of year): inf
     Crane occupancy planned (@ start of year): inf
     Crane occupancy online  (@ start of year): inf

  *** add Berth to elements
     Berth occupancy planned (after adding Berth): inf
     Berth occupancy online  (after adding Berth): inf

  *** add Quay to elements
df    Year  Terminal Capex  Maintenance  Insurance
0  2020      24334677.0          0.0    

df    Year  Truck Capex  Truck Opex
0  2020      20000.0         0.0
1  2021          0.0      2000.0
2  2022          0.0      2000.0
3  2023          0.0      2000.0
4  2024          0.0      2000.0
truck_online 3
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020      30000.0         0.0
1  2021          0.0      3000.0
2  2022          0.0      3000.0
3  2023          0.0      3000.0
4  2024          0.0      3000.0
truck_online 4
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020      40000.0         0.0
1  2021          0.0      4000.0
2  2022          0.0      4000.0
3  2023          0.0      4000.0
4  2024          0.0      4000.0
truck_online 5
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020      50000.0         0.0
1  2021          0.0      5000.0
2  2022          0.0      5000.0
3  2023          0.0      5000.0
4  2024          0.0      5000.0
truck_online 6
  *** add Truck to elements
df    Year  Truck Capex 

c:\checkouts\opentisim\opentisim\container_system_offshore_bridge.py:1955: FutureWarning:

`item` has been deprecated and will be removed in a future version

c:\checkouts\opentisim\opentisim\container_system_offshore_bridge.py:2442: FutureWarning:

`item` has been deprecated and will be removed in a future version



    Year  Truck Capex  Truck Opex
0  2020      70000.0         0.0
1  2021          0.0      7000.0
2  2022          0.0      7000.0
3  2023          0.0      7000.0
4  2024          0.0      7000.0
truck_online 8
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020      80000.0         0.0
1  2021          0.0      8000.0
2  2022          0.0      8000.0
3  2023          0.0      8000.0
4  2024          0.0      8000.0
truck_online 9
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020      90000.0         0.0
1  2021          0.0      9000.0
2  2022          0.0      9000.0
3  2023          0.0      9000.0
4  2024          0.0      9000.0
truck_online 10
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020     100000.0         0.0
1  2021          0.0     10000.0
2  2022          0.0     10000.0
3  2023          0.0     10000.0
4  2024          0.0     10000.0
truck_online 11
  *** add Truck to elements
df    Year  Truck Capex 

df    Year  Truck Capex  Truck Opex
0  2020     420000.0         0.0
1  2021          0.0     42000.0
2  2022          0.0     42000.0
3  2023          0.0     42000.0
4  2024          0.0     42000.0
truck_online 43
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020     430000.0         0.0
1  2021          0.0     43000.0
2  2022          0.0     43000.0
3  2023          0.0     43000.0
4  2024          0.0     43000.0
truck_online 44
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020     440000.0         0.0
1  2021          0.0     44000.0
2  2022          0.0     44000.0
3  2023          0.0     44000.0
4  2024          0.0     44000.0
truck_online 45
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020     450000.0         0.0
1  2021          0.0     45000.0
2  2022          0.0     45000.0
3  2023          0.0     45000.0
4  2024          0.0     45000.0
truck_online 46
  *** add Truck to elements
df    Year  Truck Ca

...

df    Year  Truck Capex  Truck Opex
0  2020    1140000.0         0.0
1  2021          0.0    114000.0
2  2022          0.0    114000.0
3  2023          0.0    114000.0
4  2024          0.0    114000.0
truck_online 115
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1150000.0         0.0
1  2021          0.0    115000.0
2  2022          0.0    115000.0
3  2023          0.0    115000.0
4  2024          0.0    115000.0
truck_online 116
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1160000.0         0.0
1  2021          0.0    116000.0
2  2022          0.0    116000.0
3  2023          0.0    116000.0
4  2024          0.0    116000.0
truck_online 117
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1170000.0         0.0
1  2021          0.0    117000.0
2  2022          0.0    117000.0
3  2023          0.0    117000.0
4  2024          0.0    117000.0
truck_online 118
  *** add Truck to elements
df    Year  Truc

df    Year  Truck Capex  Truck Opex
0  2020    1610000.0         0.0
1  2021          0.0    161000.0
2  2022          0.0    161000.0
3  2023          0.0    161000.0
4  2024          0.0    161000.0
truck_online 162
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1620000.0         0.0
1  2021          0.0    162000.0
2  2022          0.0    162000.0
3  2023          0.0    162000.0
4  2024          0.0    162000.0
truck_online 163
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1630000.0         0.0
1  2021          0.0    163000.0
2  2022          0.0    163000.0
3  2023          0.0    163000.0
4  2024          0.0    163000.0
truck_online 164
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1640000.0         0.0
1  2021          0.0    164000.0
2  2022          0.0    164000.0
3  2023          0.0    164000.0
4  2024          0.0    164000.0
truck_online 165
  *** add Truck to elements
df    Year  Truc

  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    1990000.0         0.0
1  2021          0.0    199000.0
2  2022          0.0    199000.0
3  2023          0.0    199000.0
4  2024          0.0    199000.0
truck_online 200
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2000000.0         0.0
1  2021          0.0    200000.0
2  2022          0.0    200000.0
3  2023          0.0    200000.0
4  2024          0.0    200000.0
truck_online 201
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2010000.0         0.0
1  2021          0.0    201000.0
2  2022          0.0    201000.0
3  2023          0.0    201000.0
4  2024          0.0    201000.0
truck_online 202
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2020000.0         0.0
1  2021          0.0    202000.0
2  2022          0.0    202000.0
3  2023          0.0    202000.0
4  2024          0.0    202000.0
truck_online 203
  *** add Truck 

df    Year  Truck Capex  Truck Opex
0  2020    2340000.0         0.0
1  2021          0.0    234000.0
2  2022          0.0    234000.0
3  2023          0.0    234000.0
4  2024          0.0    234000.0
truck_online 235
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2350000.0         0.0
1  2021          0.0    235000.0
2  2022          0.0    235000.0
3  2023          0.0    235000.0
4  2024          0.0    235000.0
truck_online 236
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2360000.0         0.0
1  2021          0.0    236000.0
2  2022          0.0    236000.0
3  2023          0.0    236000.0
4  2024          0.0    236000.0
truck_online 237
  *** add Truck to elements
df    Year  Truck Capex  Truck Opex
0  2020    2370000.0         0.0
1  2021          0.0    237000.0
2  2022          0.0    237000.0
3  2023          0.0    237000.0
4  2024          0.0    237000.0
truck_online 238
  *** add Truck to elements
df    Year  Truc

AttributeError: 'System' object has no attribute 'reclamation_invest'

## 3. Report all elements

In [ ]:
if True: 
    for element in OffshoreBridge.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__) # This is the dictionary containing the module's symbol table.

In [ ]:
# scroll_to_current_cell(preserve=True)

In [ ]:
# OffshoreBridge.terminal_elements_plot()

In [ ]:
# OffshoreBridge.terminal_capacity_plot()

In [ ]:
# OffshoreBridge.terminal_land_use_plot()

In [ ]:
# OffshoreBridge.storage_area_plot()

#### Add cash flow information for each of the terminal elements.

In [ ]:
terminal_capex, terminal_opex, bridge_construction, bridge_maintenance_dredging, truck_capex, truck_opex, cash_flows, cash_flows_df, cash_flows_WACC_real_df, NPV = OffshoreBridge.net_present_value()

In [ ]:
# OffshoreBridge.terminal_opex_plot(cash_flows_df)

In [ ]:
# OffshoreBridge.total_opex_plot(cash_flows_df)

In [ ]:
# print('\033[1mCash Flow Plot\033[0m')
# OffshoreBridge.cashflow_plot(cash_flows_df)

In [ ]:
scroll_to_current_cell(preserve=True)

In [ ]:
print('\033[1m    Financial Indication\033[0m')
print('  * NPV: ${}'.format(f'{round(NPV,-6):,}'))

hide_columns_list = ['Maintenance','Insurance','Energy','Labour', 'Fuel', 'Demurrage']
cash_flows_WACC_real_df.style.hide_columns(hide_columns_list).hide_index()

In [ ]:
print('\033[1m Cash Flow Plot - Weighted average cost of capital \033[0m')
OffshoreBridge.cashflow_plot(cash_flows_WACC_real_df)